<a href="https://colab.research.google.com/github/miller00315/crew_ai_studies/blob/main/task_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#!pip install crewai crewai_tools langchain_community

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

https://aistudio.google.com/apikey

https://serper.dev/api-key

https://platform.deepseek.com/api_keys


In [ ]:
import os

os.environ["SERPER_API_KEY"] = ''
os.environ["GROQ_API_KEY"] = ''

In [ ]:
from crewai import Agent, Crew, Task

In [ ]:
from crewai import LLM

llm = LLM(model="groq/llama-3.3-70b-versatile", api_key='')

https://console.groq.com/keys

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:295: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  warnings.warn(DEPRECATION_MESSAGE, DeprecationWarning)
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/scrapegraph_scrape_tool/scrapegraph_scrape_tool.py:34: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("website_url")
/usr/local/lib/python3.11/dist-packages/crewai_tools/tools/selenium_scraping_tool/selenium_scraping_tool.py:26: PydanticDeprecatedSince20: Pydantic V1 style `@validator`

In [ ]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    llm=llm,
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [ ]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    llm=llm,
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [ ]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    llm=llm,
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

# **Creating Venue Pydantic Object**

* Create a class VenueDetails using Pydantic BaseModel.

* Agents will populate this object with information about different venues by creating different instances of it.

In [ ]:
from pydantic import BaseModel
# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

# **Creating Tasks**
* By using output_json, you can specify the structure of the output you want.
* By using output_file, you can get your output in a file.
* By setting human_input=True, the task will ask for human feedback (whether you like the results or not) before finalising it.

In [ ]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [ ]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    agent=logistics_manager
)

In [ ]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

# **Creating the Crew**
**Note:** Since you set ```async_execution=True``` for ```logistics_task``` and ```marketing_task tasks```, now the order for them does not matter in the tasks list.

In [ ]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator,
            logistics_manager,
            marketing_communications_agent],

    tasks=[venue_task,
           logistics_task,
           marketing_task],

    verbose=True
)

# **Running the Crew**
* Set the inputs for the execution of the crew.

In [ ]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

**Note 1:** LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

**Note 2:**
* Since you set human_input=True for some tasks, the execution will ask for your input before it finishes running.
* When it asks for feedback, use your mouse pointer to first click in the text box before typing anything.


In [ ]:
result = event_management_crew.kickoff(inputs=event_details)

 
[2025-03-11 14:33:04][🚀 CREW 'CREW' STARTED, E1CF7E12-4642-4CBF-A133-084D0D1699FC]: 2025-03-11 14:33:04.019614
 
[2025-03-11 14:33:04][📋 TASK STARTED: FIND A VENUE IN SAN FRANCISCO THAT MEETS CRITERIA FOR TECH INNOVATION CONFERENCE.]: 2025-03-11 14:33:04.037062
 
[2025-03-11 14:33:04][🤖 AGENT 'VENUE COORDINATOR' STARTED TASK]: 2025-03-11 14:33:04.038378
# Agent: Venue Coordinator
## Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.
 
[2025-03-11 14:33:04][🤖 LLM CALL STARTED]: 2025-03-11 14:33:04.038638
 
[2025-03-11 14:33:05][✅ LLM CALL COMPLETED]: 2025-03-11 14:33:05.562980
 
[2025-03-11 14:33:05][🤖 TOOL USAGE STARTED: 'SEARCH THE INTERNET WITH SERPER']: 2025-03-11 14:33:05.563780
 
[2025-03-11 14:33:06][✅ TOOL USAGE FINISHED: 'SEARCH THE INTERNET WITH SERPER']: 2025-03-11 14:33:06.315803


# Agent: Venue Coordinator
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"tech conference venues in San Francisco\"}"
#

ERROR:root:LiteLLM call failed: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01j606df88epsb1tnas9wsd4v6` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 2778, Requested 4128. Please try again in 9.050999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}



 
[2025-03-11 14:33:11][✅ TOOL USAGE FINISHED: 'READ WEBSITE CONTENT']: 2025-03-11 14:33:11.505950


# Agent: Venue Coordinator
## Using tool: Read website content
## Tool Input: 
"{\"website_url\": \"https://ssfconf.com/ai-conference-venue-sf/\"}"
## Tool Output: 

AI Industry Conference Venue in SF - South San Francisco Conference Center
Skip to content
Meetings Overview Promotions / Packages Floor Plans Catering Services Audio/Visual Event Services Request for Proposal Media Gallery Celebrations Overview Promotions / Packages Floor Plans Catering Services Audio/Visual Event Services Ask For A Quote Media Gallery Lodging Bay Area Overview Our Community Attractions Transportation Options Contact General Info Overview Conference Center Authority Rules & Regulations Safe & Secure Events Green Initiative Privacy Policy Blog
×
Meetings Overview Promotions / Packages Floor Plans Catering Services Audio/Visual Event Services Request for Proposal Media Gallery Celebrations Overview Promotion

RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01j606df88epsb1tnas9wsd4v6` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 2778, Requested 4128. Please try again in 9.050999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}


In [ ]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)